[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/EMQA/blob/main/EN/quantlets/EMQA_hedge_compare/EMQA_hedge_compare.ipynb)

# EMQA_hedge_compare
Hedging strategy comparison: No Hedge, Full Hedge, and Optimal Hedge.
**Output:** `hedge_compare.pdf`

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

plt.rcParams.update({
    'figure.facecolor': 'none',
    'axes.facecolor': 'none',
    'savefig.facecolor': 'none',
    'savefig.transparent': True,
    'axes.grid': False,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'font.size': 11,
    'figure.figsize': (12, 6),
})

COLORS = {
    'blue': '#1A3A6E', 'red': '#CD0000', 'green': '#2E7D32',
    'orange': '#E67E22', 'purple': '#8E44AD', 'gray': '#808080',
    'cyan': '#00BCD4', 'amber': '#B5853F'
}

def save_fig(fig, name):
    fig.savefig(name, bbox_inches='tight', transparent=True, dpi=300)
    print(f"Saved: {name}")


In [ ]:
# Simulate spot price path and futures hedge
np.random.seed(42)
n_days = 252
S0 = 80  # initial spot price
mu = 0.0002
sigma = 0.02

# Spot price path
z = np.random.randn(n_days)
log_ret = mu + sigma * z
spot = S0 * np.exp(np.cumsum(log_ret))
spot = np.insert(spot, 0, S0)

# Futures price tracks spot with small basis
basis = np.random.randn(n_days + 1) * 0.3
futures = spot + basis

# Effective cost under different strategies
h_opt = 0.85  # optimal hedge ratio (estimated)

cost_no_hedge = spot
cost_full_hedge = np.full_like(spot, spot[0]) + np.cumsum(np.insert(np.diff(spot) - np.diff(futures), 0, 0))
cost_opt_hedge = spot - h_opt * (futures - futures[0])

days = np.arange(n_days + 1)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Panel 1 - Effective cost over time
axes[0].plot(days, cost_no_hedge, color=COLORS['red'], linewidth=1.5, label='No Hedge')
axes[0].plot(days, cost_full_hedge, color=COLORS['blue'], linewidth=1.5, label='Full Hedge (h=1)')
axes[0].plot(days, cost_opt_hedge, color=COLORS['green'], linewidth=1.5, label=f'Optimal Hedge (h={h_opt})')
axes[0].set_xlabel('Day')
axes[0].set_ylabel('Effective Cost ($)')
axes[0].set_title('Effective Cost Under Different Hedging Strategies')
axes[0].legend(loc='upper center', bbox_to_anchor=(0.5, -0.10), frameon=False, ncol=3)

# Panel 2 - Bar chart of cost volatility
vol_no = np.std(np.diff(cost_no_hedge))
vol_full = np.std(np.diff(cost_full_hedge))
vol_opt = np.std(np.diff(cost_opt_hedge))

labels = ['No Hedge', 'Full Hedge', 'Optimal Hedge']
vols = [vol_no, vol_full, vol_opt]
bar_colors = [COLORS['red'], COLORS['blue'], COLORS['green']]
bars = axes[1].bar(labels, vols, color=bar_colors, width=0.5, edgecolor='white')
for bar, val in zip(bars, vols):
    axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                 f'{val:.3f}', ha='center', va='bottom', fontsize=10)
axes[1].set_ylabel('Cost Volatility ($ / day)')
axes[1].set_title('Daily Cost Volatility by Strategy')

plt.tight_layout()
save_fig(fig, 'hedge_compare.pdf')
plt.show()